# Map Wrapper Development for Scipy Differential Equation

Proof of concept notebook for first Scipy, since it only requires map not the o

Error_function is a pass function.
Our "map" takes care of initiating the whole neuron group, calculating the errors and returing the value.

In [1]:
import numpy as np
import multiprocessing as mp

from scipy.optimize import differential_evolution, rosen
from scipy.optimize._differentialevolution import DifferentialEvolutionSolver


In [2]:
from brian2 import *
from brian2.equations.equations import (DIFFERENTIAL_EQUATION, Equations,
                                        SingleEquation, PARAMETER)
from brian2.input import TimedArray
from brian2 import NeuronGroup, StateMonitor, store, restore, run, defaultclock, second, Quantity
from brian2.stateupdaters.base import StateUpdateMethod



### Input / Output

In [3]:
input_traces = zeros((10,1))*volt
for i in range(1):
    input_traces[1:,i]=i*10*mV

In [4]:
# Create target current traces
output_traces = 10*nS*input_traces

In [5]:
input = input_traces
output = output_traces

### Parameters 

In [6]:
input_var = 'v'
output_var = 'I'

parameter_names = {'g', 'E'}
method = ('linear', 'exponential_euler', 'euler')
t_start = 0*second
popsize = 2
dt = 0.1 *ms
defaultclock.dt = dt

### Calculation for inner parameters

In [7]:
Nsteps, Ntraces = input_traces.shape
N = popsize * len(parameter_names)
duration = Nsteps*dt

In [8]:
N

2

### Setup The Model for Optimization
```
params, fits, error = fit_traces(model = model, input_var = 'v', output_var = 'I',\
                                 input = input_traces, output = output_traces,
                                 dt = 0.1*ms, g = [1*nS, 30*nS], E = [-20*mV,100*mV],
                                 tol = 1e-6)
```

In [9]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [10]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

INFO       No numerical integration method specified, using method 'linear' (took 0.00s). [brian2.stateupdaters.base.method_choice]


### Required Model Operations

In [11]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [12]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

In [13]:
neurons = NeuronGroup(Ntraces*N, model, method = method)
neurons.namespace['input_var'] = input_traces
neurons.namespace['output_var'] = output_traces
neurons.namespace['t_start'] = t_start
neurons.namespace['Ntraces'] = Ntraces

# Record error
neurons.run_regularly('total_error +=  (' + output_var + '-output_var(t,i % Ntraces))**2 * int(t>=t_start)',
                      when='end')

# Add the code doing the numerical integration
neurons.run_regularly(state_update_code, when='groups')

store()


In [14]:
neurons

NeuronGroup(clock=Clock(dt=100. * usecond, name='defaultclock'), when=start, order=0, name='neurongroup')

In [15]:
def error_function(params):
        # Set parameter values, duplicated over traces
        # params is a list of vectors (vector = value for population)
        d = dict()
        for name, value in zip(parameter_names, params.T):
            d[name] = (value * ones((Ntraces,N))).T.flatten()

        # Run the model
        restore()
        neurons.set_states(d, units = False)
        run(duration, namespace = {})

        e = neurons.total_error/int((duration-t_start)/defaultclock.dt)
        e = mean(e.reshape((N,Ntraces)),axis=1)
        return array(e)

### Dev Map Optimization 

#### Artificial Data

In [83]:
params = np.array([
 [ 1.80869973e-08,  2.50218013e-02],
 [ 1.88373085e-08,  9.89559934e-02]])

params.T

array([[1.80869973e-08, 1.88373085e-08],
       [2.50218013e-02, 9.89559934e-02]])

In [84]:
params2 = np.array([[ 1.12735759e-08,  8.81556360e-02],
 [ 9.04858783e-09,  8.03343489e-02],
 [ 1.60635907e-08,  1.59710651e-03],
 [ 1.66254878e-09,  2.95479385e-02],
 [ 1.80869973e-08,  2.50218013e-02],])

In [85]:
parameter_names

{'E', 'g'}

In [86]:
def error_pass(params):
    return params

In [87]:
def error_map_function(params):
    d = dict()
    for name, value in zip(parameter_names, params):
        d[name] = (value * ones((Ntraces,N))).T.flatten()
            
    return d


In [88]:
p = error_map_function(params)

In [89]:
p = error_map_function(params2)
p

{'g': array([1.12735759e-08, 8.81556360e-02]),
 'E': array([9.04858783e-09, 8.03343489e-02])}

In [78]:
neurons.get_states()

{'N': array(2),
 'i': array([0, 1], dtype=int32),
 't': 0. * second,
 'dt': 100. * usecond,
 't_in_timesteps': array(0),
 'g': array([0., 0.]) * siemens,
 'E': array([0., 0.]) * volt,
 'total_error': array([0., 0.]) * amp2}

In [73]:
def error_map_function2(func, params):
#     print('params len', len(params))
#     print('params', params)
    d = dict()
    for name, value in zip(parameter_names, params.T):
        d[name] = value
#         (value * ones((Ntraces,N))).flatten()
            
#     print('d len', len(d))
#     print('d', d)
    #### NEURONS SET_STATES HAS TO BE FIXED TO SET IT FOR ALL 
    restore()
    
    neurons.set_states(d, units = False)
    run(duration, namespace = {})

    e = neurons.total_error/int((duration-t_start)/defaultclock.dt)
    err = neurons.total_error
#     print('total err', err)
    e = mean(e.reshape((N,Ntraces)),axis=1)
    

#     print('e len', len(e))
    
    return np.array(e).T

In [74]:

res = error_map_function2(error_pass, params)
res

array([2.04818928e-19, 3.47473674e-18])

In [75]:
# res = error_map_function2(error_pass, params2)
# res

### Optimization Init

In [76]:
bounds = [(1, 30), (-20, 100)]

In [79]:
differential_evolution(error_pass, bounds, updating='deferred', workers=error_map_function2, popsize=2, disp=True)

RuntimeError: The map-like callable must be of the form f(func, iterable), returning a sequence of numbers the same length as 'iterable'

### Map Approach Test

In [81]:
def error(params):
    # Set parameter values, duplicated over traces
    # params is a list of vectors (vector = value for population)
    d = dict()
    
    print('params:', params)
    
    for name, value in zip(parameter_names, params):
        d[name] = (value * ones((Ntraces,N))).T.flatten()
            
    print('d:', d)
    
    #### NEURONS SET_STATES HAS TO BE FIXED TO SET IT FOR ALL 
    restore()
    
    neurons.set_states(d, units = False)
    run(duration, namespace = {})

    e = neurons.total_error/int((duration-t_start)/defaultclock.dt)
    e = mean(e.reshape((N,Ntraces)),axis=1)
    print(e)
    return array(e)

In [82]:
differential_evolution(error, bounds, updating='deferred', workers=map, maxiter=3)

params: [ 8.56030601 90.04418762]
d: {'g': array([8.56030601, 8.56030601]), 'E': array([90.04418762, 90.04418762])}
[0.59414158 0.59414158] kA^2
params: [16.65219159 52.53559023]
d: {'g': array([16.65219159, 16.65219159]), 'E': array([52.53559023, 52.53559023])}
[0.76533228 0.76533228] kA^2
params: [19.77020617 62.38352577]
d: {'g': array([19.77020617, 19.77020617]), 'E': array([62.38352577, 62.38352577])}
[1.52111563 1.52111563] kA^2
params: [23.53212368 44.34257077]
d: {'g': array([23.53212368, 23.53212368]), 'E': array([44.34257077, 44.34257077])}
[1.08883978 1.08883978] kA^2
params: [ 1.3357291  77.67845158]
d: {'g': array([1.3357291, 1.3357291]), 'E': array([77.67845158, 77.67845158])}
[0.01076559 0.01076559] kA^2
params: [15.6890163  -2.62869963]
d: {'g': array([15.6890163, 15.6890163]), 'E': array([-2.62869963, -2.62869963])}
[1700.87875372 1700.87875372] A^2
params: [ 6.28713079 87.30177996]
d: {'g': array([6.28713079, 6.28713079]), 'E': array([87.30177996, 87.30177996])}
[0.30

RuntimeError: The map-like callable must be of the form f(func, iterable), returning a sequence of numbers the same length as 'iterable'

# 

In [ ]:
# restore()
# neurons.set_states(p, units=False)
# e = neurons.total_error/int((duration-t_start)/defaultclock.dt)
# print(e)
# e = mean(e.reshape((N,Ntraces)),axis=1)
# e

In [ ]:
# M_out = StateMonitor(neurons, output_var, record = range(Ntraces))
# neurons.set_states(p, units = False)

# run(duration, namespace = {})
# fits = M_out.get_states()[output_var]
# fits

# error = Quantity(res.fun ** .5, dim=model.dimensions[output_var])
